In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_train_data = pd.read_parquet('../Data/credit_card_data_train.parquet').sample(150000)

In [3]:
raw_train_data.drop(columns=[raw_train_data.columns[0]],inplace=True)
raw_train_data
## useless columns -> first, last, trans_num, trans_date_trans_time, city, zip, city_pop, street
raw_train_data.drop(columns = ['first', 'last', 'trans_num', 'trans_date_trans_time', 'city', 'zip', 'city_pop', 'street'], inplace=True)
raw_train_data

,cc_num,merchant,category,amt,gender,state,lat,long,job,dob,unix_time,merch_lat,merch_long,is_fraud
1193242,3523843138706408,fraud_Cummerata-Jones,gas_transport,52.25,F,AL,32.6176,-86.9475,Drilling engineer,1970-11-20,1368418337,32.296243,-86.615674,0
269334,4742883543039288059,fraud_Rodriguez Group,gas_transport,56.46,F,HI,20.0271,-155.3697,"Scientist, research (maths)",1991-04-13,1336899812,19.571364,-155.898000,0
225541,3560725013359375,fraud_Schmitt Ltd,misc_net,9.94,F,TX,31.8599,-102.7413,Cytogeneticist,1969-09-15,1335230783,32.610754,-102.747255,0
129327,583699482621,fraud_Maggio-Fahey,misc_pos,1.74,F,WI,44.3481,-89.1101,"Engineer, civil (contracting)",1936-07-22,1331530979,44.199830,-88.494375,0
743664,3560797065840735,fraud_Miller-Hauck,grocery_pos,118.78,F,ND,46.1838,-101.2589,Film/video editor,1989-12-17,1352886074,47.138547,-101.115151,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708223,371034293500716,"fraud_Romaguera, Wehner and Tromp",kids_pets,11.76,M,CA,40.0235,-120.7986,Education administrator,1966-02-21,1351466555,40.989635,-121.696592,0
210151,2291163933867244,fraud_Huel-Langworth,misc_net,392.24,M,SC,33.9659,-80.9355,Mechanical engineer,1968-03-19,1334659947,33.755864,-79.937929,0
1128494,30199621383748,fraud_Conroy-Emard,food_dining,41.36,F,TX,33.4044,-96.2238,Sub,1977-03-23,1365955073,32.741553,-96.730234,0
478900,3586008444788268,fraud_Hamill-Daugherty,misc_net,1.04,F,WA,47.4974,-122.0107,Osteopath,1984-02-29,1343523939,47.844963,-121.732054,0


In [4]:
#state, job, dob->age,
reference_date = pd.Timestamp("2021-01-01")
raw_train_data['dob'] = pd.to_datetime(raw_train_data['dob'])
raw_train_data['dob'] = (reference_date - raw_train_data['dob']).dt.days // 365
raw_train_data.rename(columns={'dob':'age'}, inplace=True)

columns_to_be_encoded = ['state','job','category','']

In [5]:
raw_train_data

,cc_num,merchant,category,amt,gender,state,lat,long,job,age,unix_time,merch_lat,merch_long,is_fraud
1193242,3523843138706408,fraud_Cummerata-Jones,gas_transport,52.25,F,AL,32.6176,-86.9475,Drilling engineer,50,1368418337,32.296243,-86.615674,0
269334,4742883543039288059,fraud_Rodriguez Group,gas_transport,56.46,F,HI,20.0271,-155.3697,"Scientist, research (maths)",29,1336899812,19.571364,-155.898000,0
225541,3560725013359375,fraud_Schmitt Ltd,misc_net,9.94,F,TX,31.8599,-102.7413,Cytogeneticist,51,1335230783,32.610754,-102.747255,0
129327,583699482621,fraud_Maggio-Fahey,misc_pos,1.74,F,WI,44.3481,-89.1101,"Engineer, civil (contracting)",84,1331530979,44.199830,-88.494375,0
743664,3560797065840735,fraud_Miller-Hauck,grocery_pos,118.78,F,ND,46.1838,-101.2589,Film/video editor,31,1352886074,47.138547,-101.115151,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708223,371034293500716,"fraud_Romaguera, Wehner and Tromp",kids_pets,11.76,M,CA,40.0235,-120.7986,Education administrator,54,1351466555,40.989635,-121.696592,0
210151,2291163933867244,fraud_Huel-Langworth,misc_net,392.24,M,SC,33.9659,-80.9355,Mechanical engineer,52,1334659947,33.755864,-79.937929,0
1128494,30199621383748,fraud_Conroy-Emard,food_dining,41.36,F,TX,33.4044,-96.2238,Sub,43,1365955073,32.741553,-96.730234,0
478900,3586008444788268,fraud_Hamill-Daugherty,misc_net,1.04,F,WA,47.4974,-122.0107,Osteopath,36,1343523939,47.844963,-121.732054,0


In [6]:
raw_train_data['cc_num'] = np.log(raw_train_data['cc_num'])
raw_train_data.rename(columns={'cc_num':'log_cc_num'}, inplace=True)

raw_train_data['unix_time'] = np.log(raw_train_data['unix_time'])
raw_train_data.rename(columns={'unix_time':'log_unix_time'}, inplace=True)

raw_train_data['gender'] = raw_train_data['gender'].map({'M':1 , 'F':0})

In [7]:
raw_train_data

,log_cc_num,merchant,category,amt,gender,state,lat,long,job,age,log_unix_time,merch_lat,merch_long,is_fraud
1193242,35.798329,fraud_Cummerata-Jones,gas_transport,52.25,0,AL,32.6176,-86.9475,Drilling engineer,50,21.036921,32.296243,-86.615674,0
269334,43.003177,fraud_Rodriguez Group,gas_transport,56.46,0,HI,20.0271,-155.3697,"Scientist, research (maths)",29,21.013619,19.571364,-155.898000,0
225541,35.808741,fraud_Schmitt Ltd,misc_net,9.94,0,TX,31.8599,-102.7413,Cytogeneticist,51,21.012370,32.610754,-102.747255,0
129327,27.092652,fraud_Maggio-Fahey,misc_pos,1.74,0,WI,44.3481,-89.1101,"Engineer, civil (contracting)",84,21.009595,44.199830,-88.494375,0
743664,35.808761,fraud_Miller-Hauck,grocery_pos,118.78,0,ND,46.1838,-101.2589,Film/video editor,31,21.025506,47.138547,-101.115151,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708223,33.547316,"fraud_Romaguera, Wehner and Tromp",kids_pets,11.76,1,CA,40.0235,-120.7986,Education administrator,54,21.024456,40.989635,-121.696592,0
210151,35.367836,fraud_Huel-Langworth,misc_net,392.24,1,SC,33.9659,-80.9355,Mechanical engineer,52,21.011942,33.755864,-79.937929,0
1128494,31.038851,fraud_Conroy-Emard,food_dining,41.36,0,TX,33.4044,-96.2238,Sub,43,21.035120,32.741553,-96.730234,0
478900,35.815816,fraud_Hamill-Daugherty,misc_net,1.04,0,WA,47.4974,-122.0107,Osteopath,36,21.018562,47.844963,-121.732054,0


In [8]:
integer_cols = raw_train_data.select_dtypes(include=['float64','int64']).columns
print(integer_cols)

Index(['log_cc_num', 'amt', 'gender', 'lat', 'long', 'age', 'log_unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='str')


In [9]:
not_integer_cols = raw_train_data.select_dtypes(include=['str']).columns
print(not_integer_cols)

Index(['merchant', 'category', 'state', 'job'], dtype='str')


In [10]:
raw_train_data.to_parquet('../Data/cleaned_training_data.parquet',index=False)